In [1]:
import os
from lxml import etree as lxml
import zipfile, itertools
import io

In [2]:
def load_xml(z):
    with z.open("archive_manifest.xml", 'r') as f:
        tree = lxml.parse(f)
    return tree

def load_zip(file):
    return zipfile.ZipFile(file)

In [9]:
def docopy(q, z1, z2):
    for file in q:
        docopy_inner(file, z1, z2)

def docopy_inner(file, z1, z2):
    print("Copying attachment '%s'..."%file, end="")
    with z1.open(file, 'r') as src:
        z2.writestr(file, src.read())
        print("Done")

In [3]:
from scripts.tools.redmine_cache import RedmineClient
_client = None

def _get_client():
    global _client
    if _client is None:
        _client = RedmineClient()
    return _client
def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def get_issues():
    return _download_issues()

In [31]:
import datetime, pytz

def dfmt(dt):
    return dt.astimezone(pytz.timezone('US/Pacific')).strftime("%m/%d/%Y %I:%M:%S %p")

In [36]:
issues = get_issues()
issues2 = [i for i in issues.values() if i.sprint_milestone.name in ("3.1.0", "3.1.1", "Future Release") and i.tracker.name == 'Bug' and i.status.name not in ("Closed", "Rejected")]
len(issues2)

61

In [76]:
root = lxml.Element("TestTrackData", version="1638912")

for i in issues2:
    defect = lxml.SubElement(root, "defect")
    lxml.SubElement(defect, "summary").text = i.subject
    lxml.SubElement(defect, "type").text = "Defect"
    
    if i.author.name in ('James Small', 'Nathan Starkweather', 'Yas Hashimura'):
        first, last = i.author.name.split()
    else:
        if i.author.name == 'Daniel Giroux' or i.author.name == 'Christian Nilsson':
            first, last = 'James', 'Small'
        else:
            raise NameError(i.author.name)
        
    entered_by = lxml.SubElement(defect, "entered-by")
    created_by = lxml.SubElement(defect, "created-by")
    
    for e in (entered_by, created_by):
        lxml.SubElement(e, 'last-name').text = last
        lxml.SubElement(e, 'first-name').text = first
        lxml.SubElement(e, 'middle-name')  # unused
    
    lxml.SubElement(defect, 'defect-status').text = 'Defects'
    lxml.SubElement(defect, 'date-entered').text = i.created_on.strftime("%m/%d/%Y")
    lxml.SubElement(defect, "date-created").text = dfmt(i.created_on)
    lxml.SubElement(defect, "date-last-modified").text = dfmt(i.modified_on)
    
    record = lxml.SubElement(defect, 'reported-by-record')
    found_by = lxml.SubElement(record, 'found-by')
    lxml.SubElement(found_by, 'last-name').text = last
    lxml.SubElement(found_by, 'first-name').text = first
    lxml.SubElement(found_by, 'middle-name')  # unused
    
    lxml.SubElement(record, 'date-found').text = i.created_on.strftime("%m/%d/%Y")
    lxml.SubElement(record, 'description').text = i.description
    
    

In [77]:
def write_root(r, fn):
    tree = lxml.ElementTree(r)
    with open(fn, 'wb') as f:
        tree.write(f, pretty_print=True, xml_declaration=True, standalone=False, encoding='UTF-8', doctype=b'<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">')

In [78]:
write_root(root, "defect_test.xml")

In [93]:
rawxml = """<defect>
	<summary>{summary}</summary>
	<entered-by><last-name>{last_name}</last-name><first-name>{first_name}</first-name><middle-name/></entered-by>
	<date-entered>{date_entered}</date-entered>
	<type>Defect</type>
	<workaround has-formatting="true"></workaround>		
    <reported-by-record>
		<found-by><last-name>{last_name}</last-name><first-name>{first_name}</first-name><middle-name/></found-by>
		<date-found>{date_entered}</date-found>
		<description has-formatting="false">{description}</description>
		<steps-to-reproduce has-formatting="true"></steps-to-reproduce>
		<computer-config type="users"/>
		<other-hardware-and-software has-formatting="true"></other-hardware-and-software>
	</reported-by-record>
	<defect-status>Defects</defect-status>
	<created-by><last-name>{last_name}</last-name><first-name>{first_name}</first-name><middle-name/></created-by>
	<date-created>4/5/2019 3:49:28 PM</date-created>
	<creation-method>Add window</creation-method>
	<last-modified-by><last-name>{last_name}</last-name><first-name>{first_name}</first-name><middle-name/></last-modified-by>
	<date-last-modified>{long_date}</date-last-modified>
</defect>"""

raw = []
from xml.sax.saxutils import escape as xesc
for i in issues2:
    if i.author.name in ('James Small', 'Nathan Starkweather', 'Yas Hashimura'):
        first, last = i.author.name.split()
    else:
        if i.author.name == 'Daniel Giroux' or i.author.name == 'Christian Nilsson':
            first, last = 'James', 'Small'
        else:
            raise NameError(i.author.name)
    desc = i.description.replace("\r\n", "\n")  # .replace("\r", "&#13;").replace("\n", "&#10;")
    x = rawxml.format(summary=xesc(i.subject), last_name=last, first_name=first, description=xesc(desc), 
                      date_entered=i.created_on.strftime("%m/%d/%Y"), long_date=dfmt(i.created_on))
    raw.append(x)

In [94]:
root = lxml.Element("TestTrackData", version="1638912")
for x in raw:
    try:
        e = lxml.fromstring(x)
        root.append(e)
    except:
        print(x)
        raise
        
write_root(root, 'test.xml')

In [95]:
with open("XML_Export.xml", 'rb') as f:
    print(f.read())

b'<?xml version="1.0" encoding="UTF-8" standalone="no"?>\r\n<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">\r\n<TestTrackData version="1638912">\r\n\r\n<defect>\r\n\t<record-id>123</record-id>\r\n\t<defect-number>123</defect-number>\r\n\t<summary>Problems with "Configure Users" tab</summary>\r\n\t<entered-by><last-name>Small</last-name><first-name>James</first-name><middle-name></middle-name></entered-by>\r\n\t<date-entered>7/9/2018</date-entered>\r\n\t<type>Defect</type>\r\n\t<workaround has-formatting="false"></workaround>\t\t<custom-field-value field-name="Risk" field-value="" />\r\n\r\n\t<custom-field-value field-name="Acceptance Criteria">\r\n\r\n\t\t<multi-line-text has-formatting="false"></multi-line-text>\r\n\t</custom-field-value>\t\t<custom-field-value field-name="Days Open to Close" field-value=" Days" />\r\n\r\n\t<reported-by-record>\r\n\t\t<found-by><last-name>Small</last-name><first-name>James</first-name><middle-name></middle-name></found-by>\r\n\t\t<date-found>7/9/2

In [67]:
def write_root(newroot, attachments, name):
    newtree = lxml.ElementTree(newroot)
    
    with io.BytesIO() as f:
        newtree.write(f, pretty_print=True, xml_declaration=True, standalone=False, encoding='UTF-8', doctype=b'<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">')
        data = f.getvalue()
    
    with zipfile.ZipFile("Helix Migration Test\\%s.zip"%name, 'w', zipfile.ZIP_DEFLATED) as newzip:
        for fp in attachments:
            docopy_inner(fp, thezip, newzip)
        
        newzip.writestr("archive_manifest.xml", data)

def create_zip_file_for(root, type, name):
    newroot = _simple_copy(root)
    for e in root.xpath("%s"%type):
        _deep_copy(newroot, e)

    attachments = []
    for attachment in newroot.xpath("%s/*[local-name()='attachment' or local-name()='tcm-attachment']"%type):
        fp = attachment.attrib['filespec']
        attachments.append(fp)
        
    write_root(newroot, attachments, name)
        
#     with open("Helix Migration Test\\archive_manifest_%s.xml"%type, 'wb') as f:
#         f.write(data)
    

In [34]:
#create_zip_file_for(uc.root, 'requirement', 'RequirementImport')
create_zip_file_for(uc.root, 'test-case', 'TestCaseImport')
#create_zip_file_for(uc.root, 'requirement-document', 'RequirementDocumentImport')
create_zip_file_for(uc.root, 'defect', 'DefectImport')

Copying attachment '00066982.dat'...Done
Copying attachment '00067000.dat'...Done
Copying attachment '00066999.dat'...Done
Copying attachment '00066984.dat'...Done


In [35]:
def create_zip_root_docs(root):
    newroot = _simple_copy(root)
    seen = set()
    for rid in root.xpath("requirement-document/document-tree-node/node-requirement-id/text()"):
        if rid not in seen:
            seen.add(rid)
            e = themap[rid]
            e = copy_element(e)
            newroot.append(e)
        
    for req in root.xpath("requirement"):
        rid = req.find("record-id").text
        if rid not in seen:
            seen.add(rid)
            _deep_copy(newroot, req)
    
    for doc in root.xpath("requirement-document"):
        _deep_copy(newroot, doc)
    
    attachments = []
    for attachment in newroot.xpath("requirement/*[local-name()='attachment' or local-name()='tcm-attachment']"):
        fp = attachment.attrib['filespec']
        attachments.append(fp)
        
    write_root(newroot, attachments, "RequirementDocumentImport")
        
#     with open("Helix Migration Test\\archive_manifest_requirement-document.xml", 'wb') as f:
#         f.write(data)

In [36]:
create_zip_root_docs(uc.root)

Copying attachment '00066972.dat'...Done
Copying attachment '00066971.dat'...Done
Copying attachment '00066973.dat'...Done
Copying attachment '00066974.dat'...Done
Copying attachment '00066975.dat'...Done
Copying attachment '00066978.dat'...Done
Copying attachment '00066977.dat'...Done
Copying attachment '00066976.dat'...Done
Copying attachment '00066990.dat'...Done
Copying attachment '00066980.dat'...Done
Copying attachment '00066994.dat'...Done
Copying attachment '00066993.dat'...Done
Copying attachment '00066997.dat'...Done
Copying attachment '00066996.dat'...Done
Copying attachment '00066995.dat'...Done
